# LEARNING PYTORCH WITH EXAMPLES

- 원본 : [링크](https://pytorch.org/tutorials/beginner/pytorch_with_examples.html)
- Author : [Justin Johnson](https://github.com/jcjohnson/pytorch-examples)
- 번역 & 주석 추가 : [박범진](https://github.com/pbj0812)

# Autograd

## PyTorch: Tensors and autograd

위 예제(tensor)에서, 우리는 뉴럴 네트워크의 forward와 backward에 대한 설계를 직접 해줘야 했다. 작은 two-layer 구성의 네트워크에서 직접 설계하는 것은 힘든 일은 아니지만, 복잡하고 큰 네트워크에서는 쉬운 일이 아니다.

다행스럽게도, 우리는 네트워크에서의 backward 연산을 자동 미분을 이용하여 계산할 수 있다. PyTorch에서 autograd 패키지는 정확하게 이 기능을 제공하고 있다. autograd를 사용할 때, 네트워크의 forward 연산은 computational graph에 정의된다; 그래프에서 node는 Tensor가 되고, edge는 입력 Tensor에서 생산되는 출력 Tensor에 대한 함수가 된다. 이 그래프에 의한 backpropagating은 gradients의 연산을 쉽게 만들어 준다.

이 말은 복잡하게 들리기에, 간단히 예를 들어본다. 각각의 Tensor는 computational graph에서의 노드로 표현된다. 만약 x Tensor가 x.requires_grad=True를 가지고 있을 경우 x.grad는 x의 gradient를 가지게 된다.

이번에 우리는 PyTorch Tensor와 autograd를 사용하여 two-layer로 이루어진 네트워크를 구현해본다; 이제 우리는 더이상 network내의 backward 연산에 대한 직접적인 구현은 하지 않아도 된다:

In [1]:
# -*- coding: utf-8 -*-
import torch

In [2]:
dtype = torch.float
# device = torch.device("cpu") # GPU 사용 불가시 주석을 푼다.
device = torch.device("cuda:0") # GPU 사용 가능시 주석을 푼다.

In [3]:
N, D_in, H, D_out = 64, 1000, 100, 10

# input과 output을 가지는 random한 Tensor 생성
# requires_grad=False 는 Tensor에 대한 backward 연산시 gradients를 계산하지 않는 옵션이다.

x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# weight에 대한 random한 Tensor 생성
# requires_grad=True 는 Tensor에 대한 backward 연산시 gradients를 계산하는 옵션이다.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6

In [4]:
since = time.time()
for t in range(10000):
    # Forward pass 는 기존의 방식과 같지만, backward 연산에 대해서는 손으로 풀지 않아도 된다.
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    # Tensor를 통한 loss 연산
    # loss의 형태는 (1,)
    # loss.item()은 loss에서의 스칼라 값을 가진다.
    loss = (y_pred - y).pow(2).sum()
    
    if t%100 == 0:
        print(t, loss.item())
    # backward 연산을 위한 autograd 사용. 이 구문은 requires_grad=True라고 지정된 모든 Tensor에 대한 
    # loss의 gradient를 계산한다.
    # 이후 w1.grad와 w2.grad는 w1와 w2 각각에 대한 loss의 gradient를 가진다.
    loss.backward()
    
    # 수동으로 weight를 갱신하기 위하여 torch.no_grad()를 사용한다.
    # 왜냐하면 weight들은 requires_grad=True를 가지지만, 우리는 autograd 내에서 추적할 필요가 없기 때문이다.
    # 대체적인 방법으로는 weight.data와 weight.grad.data.를 사용하는 방법이 있다.
    # tensor.data는 tensor를 저장하는 저장소를 제공하지만, 이력을 추적하지는 않는다.
    # torch.optim.SGD 를 사용할 수도 있다.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # weight 갱신후 수동 초기화
        w1.grad.zero_()
        w2.grad.zero_()
        
now = time.time()

0 33383432.0
100 822.190673828125
200 8.993412971496582
300 0.18210481107234955
400 0.004567751660943031
500 0.0003030354855582118
600 6.236392800929025e-05
700 2.5421461032237858e-05
800 1.5219056876958348e-05
900 1.062655974237714e-05
1000 8.182493729691487e-06
1100 6.5120411818497814e-06
1200 5.386174962040968e-06
1300 4.668338078772649e-06
1400 4.064198037667666e-06
1500 3.5892840060114395e-06
1600 3.183968601661036e-06
1700 2.9025666208326584e-06
1800 2.6311072360840626e-06
1900 2.39645646615827e-06
2000 2.2050201096135424e-06
2100 2.044190750893904e-06
2200 1.922990804814617e-06
2300 1.7555100839672377e-06
2400 1.6076536439868505e-06
2500 1.5479515695915325e-06
2600 1.4235063190426445e-06
2700 1.3222905863585765e-06
2800 1.248449734703172e-06
2900 1.1970320201726281e-06
3000 1.143174358730903e-06
3100 1.0892467798839789e-06
3200 1.0330104487366043e-06
3300 9.901873454509769e-07
3400 9.474897524341941e-07
3500 9.101531759370118e-07
3600 8.716738193470519e-07
3700 8.510766065228381

In [5]:
print(now - since)

15.585309267044067
